In [30]:
import numpy as np
import graspy as gp
import matplotlib.pyplot as plt
from graspy.cluster import AutoGMMCluster
from itertools import product
from joblib import Parallel, delayed
import pickle
%matplotlib inline

from sklearn.metrics import adjusted_rand_score

In [31]:
a = 0.01
b = 0.02
c = 0.01
d = 0.002
e = 0.1
f = 0.002
g = 0.015

# a = 0.5
# b = a / 2
# c = a / 20
# d = a 
# e = 1.5 * a
# f = 1.25 * a
# g = a / 2

B = np.array([
    [a, b, c, d],
    [b, e, f, g],
    [c, f, a, b],
    [d, g, b, e]
])

projection1 = np.array([
    [np.mean(B[:2,:2]), np.mean(B[:2,2:])],
    [np.mean(B[:2,2:]), np.mean(B[2:,2:])],
])
print(projection1)

projection2 = np.array([
    [np.mean([a, c, c, a]), np.mean([b, d, f, b])],
    [np.mean([b, d, f, b]), np.mean([e, g, g, e])]
])

print(projection2)

[[0.0375  0.00725]
 [0.00725 0.0375 ]]
[[0.01   0.011 ]
 [0.011  0.0575]]


In [32]:
def experiment(n, rep, B, p=0.25, acorn=None):
    if acorn != None:
        np.random.seed(acorn)
    
    score = np.zeros(4,dtype=float)
    pi = np.array([p, p, p, p])
    A = gp.simulations.sbm((n*pi).astype(int), B)
    
    affinity_labels = np.array(['L'] * (n//2) + ['R'] * (n//2)) #left right
    core_labels = np.tile(["G"] * (n//4) + ["W"] * (n//4), 2) # gray white
    
    lse = gp.embed.LaplacianSpectralEmbed().fit_transform(
    gp.utils.augment_diagonal(A)
    )
    ase = gp.embed.AdjacencySpectralEmbed().fit_transform(A)

    affinity = ['euclidean','manhattan','none']
    lse_gmm = AutoGMMCluster(2, 2, affinity=affinity).fit_predict(lse)
    ase_gmm = AutoGMMCluster(2, 2, affinity=affinity).fit_predict(ase)
    
    score[0] = adjusted_rand_score(lse_gmm, affinity_labels)
    score[1] = adjusted_rand_score(lse_gmm, core_labels)
    score[2] = adjusted_rand_score(ase_gmm, affinity_labels)
    score[3] = adjusted_rand_score(ase_gmm, core_labels)
    
    with open('../../../res/res'+str(n)+'_'+str(rep)+'.pickle','wb') as f:
        pickle.dump(score,f)

In [34]:
nodes = range(8000,11000,1000)
reps = range(1,101,1)
iterable = product(nodes,reps)

Parallel(n_jobs=-2,verbose=1)(delayed(experiment)(n, rep, B, acorn=n*rep) for n,rep in iterable)


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 55 concurrent workers.


MemoryError: Unable to allocate array with shape (8000, 8000) and data type float64